In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16
import h5py
import os



In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 18047
nb_val_samples = 4377
image_size = (224,224)
batch_size = 128
class_num = 10

In [13]:
def build_simple_convnet(input_shape=(224,224,3),class_num=class_num):
    
    model = Sequential()
    model.add(Conv2D(32,(3,3),input_shape=input_shape,padding='valid',activation='relu'))
    model.add(MaxPool2D())
    
    model.add(Conv2D(64,(3,3),padding='valid',activation='relu'))
    model.add(MaxPool2D())
    
    model.add(Conv2D(128,(3,3),padding='valid',activation='relu'))
    model.add(MaxPool2D())
    
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(class_num,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [4]:
def generate_batches_data(dir_path, 
                    gen=image.ImageDataGenerator(), 
                    target_size=(224,224), 
                    batch_size=32, 
                    class_mode='categorical', 
                    shuffle=False):
    
    return gen.flow_from_directory(dir_path, 
                                   target_size=target_size, 
                                   batch_size=batch_size,
                                   class_mode=class_mode, 
                                   shuffle=shuffle)

In [5]:

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=5,
                                         horizontal_flip=True,
                                         fill_mode='nearest')

validation_datagen = image.ImageDataGenerator(rescale=1./255)

# test_datagen = image.ImageDataGenerator(rescale=1./255)


train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=train_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=validation_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [14]:
model = build_simple_convnet(input_shape=(224,224,3))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 86528)             0         
__________

In [15]:
model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=10,
                    callbacks=[TensorBoard()],
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // batch_size)

model.save('simple_cnn_model.h5')

Epoch 1/10
140/140 [==============================] - 299s 2s/step - loss: 2.1405 - acc: 0.2149 - val_loss: 2.4901 - val_acc: 0.3024
Epoch 2/10
140/140 [==============================] - 305s 2s/step - loss: 1.6196 - acc: 0.4065 - val_loss: 2.5012 - val_acc: 0.3893
Epoch 3/10
140/140 [==============================] - 301s 2s/step - loss: 1.2750 - acc: 0.5356 - val_loss: 2.4390 - val_acc: 0.3213
Epoch 4/10
140/140 [==============================] - 300s 2s/step - loss: 1.0741 - acc: 0.6135 - val_loss: 2.3079 - val_acc: 0.3839
Epoch 5/10
140/140 [==============================] - 297s 2s/step - loss: 0.9316 - acc: 0.6658 - val_loss: 1.9763 - val_acc: 0.3799
Epoch 6/10
140/140 [==============================] - 296s 2s/step - loss: 0.8171 - acc: 0.7116 - val_loss: 2.7149 - val_acc: 0.3700
Epoch 7/10
140/140 [==============================] - 293s 2s/step - loss: 0.7201 - acc: 0.7485 - val_loss: 2.4409 - val_acc: 0.3921
Epoch 8/10
140/140 [==============================] - 296s 2s/step - 

In [9]:
#load VGG16 model
vgg16_base_model = VGG16(weights='imagenet', include_top=False)
vgg16_base_model.summary()


553467904/553467096 [==============================] - 6s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_po

In [ ]:
datagen = image.ImageDataGenerator(rescale=1./255)
train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=False)

validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=validation_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

#get the lables
train_labels = train_generator.classes
validation_labels = validation_generator.classes

In [ ]:
#save the train bottleneck features
vgg16_bottleneck_features_train = vgg16_base_model.predict_generator(train_generator, 
                                                          nb_train_samples // batch_size)
np.save(open('vgg16_bottleneck_features_train.npy', 'w'),
        vgg16_bottleneck_features_train)

#save the validation bottleneck features
vgg16_bottleneck_features_validation = vgg16_base_model.predict_generator(validation_generator, 
                                                               nb_val_samples // batch_size)
np.save(open('vgg16_bottleneck_features_validation.npy', 'w'),
        vgg16_bottleneck_features_validation)

In [ ]:
#train the top model
train_data = np.load(open('vgg16_bottleneck_features_train.npy'))
validation_data = np.load(open('vgg16_bottleneck_features_validation.npy'))

top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(class_num, activation='softmax'))

top_model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

top_model.fit(train_data, train_labels,
              epochs=50,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
              shuffle=False)
top_model.save_weights('vgg16_top_model_fc.h5')

### checkpoint

In [ ]:
vgg16_model = VGG16(weights='imagenet', include_top=False)

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(class_num, activation='softmax'))
top_model.load_weights('vgg16_top_model_fc.h5')

vgg16_model.add(top_model)
vgg16_model.summary()
for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name)

In [ ]:

train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
                                         height_shift_range=0.1,
                                         width_shift_range=0.1,
                                         channel_shift_range=5,
                                         horizontal_flip=True,
                                         fill_mode='nearest')

validation_datagen = image.ImageDataGenerator(rescale=1./255)


train_generator = generate_batches_data(
                                    dir_path=train_path,
                                    gen=train_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = generate_batches_data(
                                    dir_path=validation_path,
                                    gen=validation_datagen,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

#frozen the first 15 layers of vgg16
for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
vgg16_model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                      metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        samples_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        nb_val_samples=nb_validation_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),CSVLogger('vgg16_fine_tune')])

In [ ]:
# for layer in vgg16_model.layers[15:25]:
#     layer.trainable = True

# vgg16_model.compile(optimizer=SGD(lr=1e-4,momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])
# vgg16_model.fit_generator(train_generator,
#                     steps_per_epoch=nb_train_samples // batch_size,
#                     epochs=50,
#                     callbacks=[TensorBoard()],
#                     validation_data=validation_generator,
#                     validation_steps=nb_val_samples // batch_size,
#                     callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),CSVLogger('vgg16_fine_tune')])

# vgg16_model.save_weights('vgg16_fine_tuning_weights.h5')